## 历史以及基操
正则表达式 -> 上下文无关语法  -> 统计学派+规则学派 -> 隐马尔可夫  -> 循环神经网络  -> 深度学习有效性  -> Word2Vec（词向量）

### 歧义

- 词法分析歧义、语法分析歧义、语义分析歧义、NLP实际应用歧义（如语音识别）     
口语处理、分词处理、新词语产生、上下文知识、实体词   
***
- 1.词汇表， 字符表
- 2.句子或词语概率分布等级并由此引出字后面某字的概率（贝叶斯链式法则）

### 语言模型

- 一阶马尔科夫模型（当前随机变量取值只与上一时刻有关）
    - 句子中某个字只与前面一个字或词有关
- 二阶马尔科夫模型（当前随机变量取值只与上两时刻有关）
    - 句子中某个字只与前面2个字或词有关
- 可变长序列（句子长度是随机），定义停止状态符号和开始状态符号
- 三元语言模型 (N^3)   

```
句子： **我是中国人<s>
模型： p(我是中国人<s>)=q(我|*，*)q(是|我，*)...q(<s>|国，人)
```
- 极大似然估计 
```
参数： q(w|u,v)                  
MLE: q(w|u,v) = Count(w,u,v) / Count(u,v) -> q(人|国,中) = Count(中,国,人) / Count(国,人)   
```
- EM算法  
```
```
- 语言模型评价：迷惑度      
    - 测试集： m个句子$s_1,s_2,...,s_m$             
    - log概率：    大
        $$log \prod_{i=0}^mp(s_i) = \sum_{i=1}^mlog(p(s_i))$$           
    - perplexity： 小
        $$Perplexity = 2^{-l}$$   
        $$l = \frac{1}{M}\sum_{i=1}^mlog(p(s_i)),M是测试集总词量$$ 

***
- 稀疏性问题（语料库有限，语料库不存在某个词而导致实体词等被划分）    
    - 线性插值平滑
        - 高元n-gram模型没有足够数据时，改用低元n-gram模型进行概率估计         
        $$\hat{q}(w|u,v) = \lambda_3q(w|u,v) + \lambda_2q(w|u) + \lambda_1q(w)$$
        $$sum_i\lambda_i = 1$$           
        - hold-out数据交叉检验
    - 拉普拉斯平滑
        $$q(w|u) = \frac{c(w|u)+1}{c(w)+L}$$(L是所有训练集biggram的个数)
    
-  解码问题
    - 动态规划（?）
    
    - Viterbi算法
   ![avatar](../Img/viterbi.png)
   ![avatar](../Img/viterbi_show.png)

***
- 隐马尔卡夫模型HMM ---> 序列到序列的问题
    - 词性标注        
     ```
     in: 我是中国人       
     out: 我/n是/v中国人/n 
     ```
    - 命名实体识别(NER)          
    ```
    in: 大摩维持阿里巴巴的增持评级               
    out: 大摩/[Company]维持阿里巴巴/[Company]的增持评级
    ```
    - 模型参数        (参数数量： $M^2$ + $NM$)       
    $$词性序列二元马尔科夫链：q(s|u) (s\in S \bigcup \{<S>\}, u\in S \bigcup \{*\})---> M^2$$      
    $$词性与词语条件概率：e(x|s) (s\in S, x\in V) ---> NM$$  

### 神经语言模型 -词向量
参看论文：[A Neural Probabilistic Language Model, Yoshua Bengio,etc](https://wenku.baidu.com/view/82925cbba76e58fafbb0030a.html)        
解释文章：[A Neural Probabilistic Language Model 论文阅读及实战](https://www.jianshu.com/p/be242ed3f314)      
![avatar](../Img/NLM.png)

### 递归神经网络 RNN
参看论文： [Bidirection LSTM-CRF Models for Sequence Tagging, Zhiheng Huang, etc](https://www.docin.com/p-1652883441.html)       
解释文章： [【论文学习】Bidirectional LSTM-CRF Models for Sequence Tagging（论文翻译）](https://blog.csdn.net/Elvira521yan/article/details/88415512)              
![avatar](../Img/LSTM.png)

##  操作

In [1]:
import jieba
import re                                         # 寻找中文句子
import copy                                       # 类的复制
from sklearn.preprocessing import LabelEncoder    # 映射为整数
from collections import OrderedDict

import numpy as np
import pandas as pd

In [52]:
with open('../Data/DRM.txt','rb')as f:
    print(f.read().decode('utf-8')[:300])

《红楼梦》曹雪芹

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何关系。
在线阅读：http://www.shuyaya.com/read/2034/
--------------------------------------------------

第一回  甄士隐梦幻识通灵　贾雨村风尘怀闺秀

    		

    此开卷第一回也．作者自云：因曾历过一番梦幻之后，故将真事隐去，而借"通灵"之说，撰此《石头记》一书也．故曰"甄士隐"云云．但


### 分句

In [66]:
# 断句
lst_sectences = []                                                   # 文章划分为句子的列表
# str_puntCode = '.!?~． 。！？～\n'                                   # 句子终结符号串，判断句子是否结束
str_puntCode = "。！？!?\n\r"
start,i  = 0, 1

with open('../Data/DRM.txt','rb')as f:
    str_Text = f.read().decode('utf-8')
    str_Text += 's'
    assert type(str_Text)==str, "'str_Text' is not string"      # 检查正文读取的格式
    for i in range(len(str_Text)-1):
        # 句子结束
        if str_Text[i] in str_puntCode and str_Text[i+1] not in str_puntCode:
            lst_sectences.append(str_Text[start:i+1].strip())
            start = i+1                                                  # 开始指针指向结束符后，新起一句
            i += 1
        # 有内容
        else:
            i += 1
            
lst_sectences[:5]

['《红楼梦》曹雪芹',
 '严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何关系。',
 '在线阅读：http://www.shuyaya.com/read/2034/',
 '--------------------------------------------------',
 '第一回  甄士隐梦幻识通灵\u3000贾雨村风尘怀闺秀']

In [70]:
# 只保留中文并存储到新文件中
filter_re = re.compile(u'[^\u4E00-\u9FA5]')            # 只保留中文

with open('../Data/cleaned_DRM.txt', 'w') as f:
    for s in lst_sectences:
        cleaned_s = filter_re.sub(r'', s)             # 裁剪中文句子
        if len(cleaned_s) :
            f.write(cleaned_s+'\n')

In [73]:
# 检查实际文本
i = 0
with open('../Data/cleaned_DRM.txt', 'r') as f:
    for l in f:
        if i >5:  break
        print(l)
        i += 1

红楼梦曹雪芹

严正声明本书为丫丫小说网的用户上传至其在本站的存储空间本站只提供全集电子书存储服务以及免费下载服务以下作品内容之版权与本站无任何关系

在线阅读

第一回甄士隐梦幻识通灵贾雨村风尘怀闺秀

此开卷第一回也作者自云因曾历过一番梦幻之后故将真事隐去而借通灵之说撰此石头记一书也故曰甄士隐云云但书中所记何事何人

自又云今风尘碌碌一事无成忽念及当日所有之女子一一细考较去觉其行止见识皆出于我之上何我堂堂须眉诚不若彼裙钗哉



### 语言模型